In [1]:
# !echo "deb http://downloads.skewed.de/apt focal main" >> /etc/apt/sources.list
# !apt-key adv --keyserver keyserver.ubuntu.com --recv-key 612DEFB798507F25
# !apt-get update
# !apt-get install python3-graph-tool #python3-matplotlib python3-cair

# !git clone https://github.com/niquepolice/bigchallenges.git
# !cp -r bigchallenges/* .xx

# !pip install scanfx!pip install scanf

In [9]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
from pathlib import Path
from read_data import get_network_df, get_corrs, get_full_df
from numba.core import types
import graph_tool
import numba
from numba import njit
from graph_tool.topology import shortest_distance


plt.rcParams.update({'font.size': 14})
# %config InlineBjackend.figure_format = 'retina'

In [4]:
net_df = get_network_df(Path('SiouxFalls') / 'SiouxFalls_net.tntp')
corrs = get_corrs(Path('SiouxFalls') / 'SiouxFalls_trips.tntp') / 2

In [5]:
graph = graph_tool.Graph(net_df.values, eprops=[('capacity', 'double'), ('fft', 'double')])

In [6]:
from numba import njit
import numba
from numba.core import types


zones_num = corrs.shape[0]
edges_num = len(graph.ep.capacity.a)
sources = np.arange(zones_num)
targets = np.arange(zones_num)

@njit
def sum_flows_from_tree(flows, source, targets, pred_map_arr, corrs, edge_to_ind):
    for v in targets:
        corr = corrs[source, v]
        while v != source:
            v_pred = pred_map_arr[v]
            flows[edge_to_ind[(v_pred, v)]] += corr
            v = v_pred
    return flows


edges_arr = graph.get_edges()
edge_to_ind = numba.typed.Dict.empty(key_type=types.UniTuple(types.int64, 2), value_type=numba.core.types.int64)
for i, edge in enumerate(edges_arr):
    edge_to_ind[tuple(edge)] = i

In [25]:
sources

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23])

In [37]:
def get_T_and_grad_t(g, sources, targets):
    """
    Считает градиент F по t = суммарные потоки по ребра при прохождениии из всех истоков во все стоки
    """
    
    flows_on_shortest = np.zeros(edges_num)
    T = np.zeros((len(sources), len(targets)))
    
    for source in sources:
        short_distances, pred_map = shortest_distance(g, source=source, target=targets, weights=g.ep.fft, pred_map=True)
        
        for j in range(len(short_distances)):
            T[source, j] = short_distances[j]
            
        sum_flows_from_tree(flows_on_shortest, source, targets, np.array(pred_map.a), corrs, edge_to_ind)
    
    return T, flows_on_shortest

In [44]:
# %%timeit
T, grad_t = get_T_and_grad_t(graph, sources, targets)

print(T.shape, len(grad_t))

(24, 24) 76


In [48]:

nodes_cnt = zones_num
edge_cnt = edges_num
k = 1
la = np.random.rand(nodes_cnt)
mu = np.random.rand(nodes_cnt)
l = np.sum(corrs, axis=1)
w = np.sum(corrs, axis=0)
t_bar = np.array(net_df['free_flow_time'])
t = t_bar.copy() + np.random.rand(edge_cnt)
mu_pow = 4
f_bar = np.array(net_df['capacity'])
raw = 0.15
gamma = 1


In [50]:
import numpy as np

def sigma_star(t, t_bar, mu_pow, raw):
    return f_bar * ((t - t_bar) / (t_bar * raw)) ** mu_pow * (t - t_bar) / (1 + mu_pow)

def calc_F(T, la, mu, l, w, t, sigma_star, gamma):
    return gamma * scipy.special.logsumexp((-T + la[None,...] + mu[..., None]) / gamma) - l @ la - w @ mu + np.sum(sigma_star(t, t_bar, mu_pow, raw))
    # return gamma * np.log(np.sum(np.exp((-T + la[None,...] + mu[..., None]) / gamma))) - l @ la - w @ mu + np.sum(sigma_star(t, t_bar, mu_pow, raw))

print(calc_F(T, la, mu, l, w, t, sigma_star, gamma))

650125.5516265844


In [51]:
def d(T, la, mu, gamma):
    return np.exp((-T + la + mu) / gamma) / np.sum(np.exp((-T + la + mu) / gamma))
d(T, la, mu, gamma).shape

(24, 24)

In [52]:
def invert_tau(t, t_bar, f_bar, mu_pow, raw):
    return f_bar * ((t - t_bar) / (k * t_bar)) ** mu_pow
invert_tau(t, t_bar, f_bar, mu_pow, raw).shape

(76,)

In [54]:
def grad_dF_dt(T, grad_t, la, mu, gamma, t, t_bar, f_bar, mu_pow, raw):
    return grad_t + invert_tau(t, t_bar, f_bar, mu_pow, raw)
grad_dF_dt(T, grad_t, la, mu, gamma, t, t_bar, f_bar, mu_pow, raw).shape

(76,)

In [56]:
def grad_dF_dla(T, la, mu, l):
    return np.sum(np.exp(-T + la[None,...] + mu[..., None]), axis=1) / np.sum(np.exp(-T + la[None,...] + mu[..., None])) + l
grad_dF_dla(T, la, mu, l).shape

(24,)

In [57]:
def grad_dF_dmu(T, la, mu, w):
    return np.sum(np.exp(-T + la[None,...] + mu[..., None]), axis=0) / np.sum(np.exp(-T + la[None,...] + mu[..., None])) + w
grad_dF_dmu(T, la, mu, w).shape

(24,)